In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import os, sys, json


In [ ]:
negative_result_folder = 'positive/data'
files = os.listdir(negative_result_folder)
df_all = pd.DataFrame()
for file in files:
    name = file.split('.')[0]
    filter_setting = name.split('_')[2]
    acc_setting = name.split('_')[5]
    gyro_setting = name.split('_')[8]
    file_count = name.split('_')[9]
    df = pd.read_csv(os.path.join(negative_result_folder, file))
    df = df.drop(columns=['Unnamed: 0'])
    # display(df, filter_setting, acc_setting, gyro_setting, file_count)
    #add filter setting, acc setting, gyro setting, file count to df
    df['filter_setting'] = filter_setting
    df['acc_setting'] = acc_setting
    df['gyro_setting'] = gyro_setting
    df['file_count'] = file_count
    df_all = pd.concat([df_all, df], ignore_index=True)
display(df_all)

In [ ]:
#plot the histogram per filter setting and acc setting
df_all['filter_setting'] = df_all['filter_setting'].astype('category')
df_all['acc_setting'] = df_all['acc_setting'].astype('category')

#plot only 16g sorted by filter setting
# df = df_all[df_all['acc_setting'] == '16G']
# df = df.sort_values(by=['filter_setting'])
# fig = px.histogram(df, x="acc_X", y=df.index, color="filter_setting", marginal="rug", hover_data=df.columns)
# fig.update_layout(title_text='Histogram of acc_X for 16g')
# fig.show()

plot_folder = 'positive/plots'

for acc_setting in df_all['acc_setting'].unique():
    df = df_all[df_all['acc_setting'] == acc_setting]
    df = df.sort_values(by=['filter_setting'])
    fig = px.histogram(df, x="acc_X", y=df.index, color="filter_setting", marginal="rug", hover_data=df.columns)
    #update x axis title
    fig.update_xaxes(title_text='acceleration x int 16')
    fig.update_yaxes(title_text='sum of samples')
    fig.update_layout(title_text='Histogram of acc_X for acc setting: ' + acc_setting)
    fig.show()
    
# for filter_setting in df_all['filter_setting'].unique():
#     for acc_setting in df_all['acc_setting'].unique():
#         df = df_all[(df_all['filter_setting'] == filter_setting) & (df_all['acc_setting'] == acc_setting)]
#         fig = px.histogram(df, x="acc_X", y=df.index, color="acc_setting", marginal="rug", hover_data=df.columns)
#         fig.update_layout(title_text='Histogram of acc_X for filter setting: ' + filter_setting + ' and acc setting: ' + acc_setting)
#         fig.show()
#         break
#     break

# for acc_setting in df_all['acc_setting'].unique():
#     for filter_setting in df_all['filter_setting'].unique():
#         df = df_all[(df_all['filter_setting'] == filter_setting) & (df_all['acc_setting'] == acc_setting)]
#         fig = px.histogram(df, x="acc_X", y=df.index, color="acc_setting", marginal="rug", hover_data=df.columns)
#         fig.update_layout(title_text='Histogram of acc_X for filter setting: ' + filter_setting + ' and acc setting: ' + acc_setting)
#         fig.show()
#     break

In [ ]:
#plot the histogram per filter setting and gyr setting
df_all['gyro_setting'] = df_all['gyro_setting'].astype('category')
df_all['filter_setting'] = df_all['filter_setting'].astype('category')

for gyro_setting in df_all['gyro_setting'].unique():
    df = df_all[df_all['gyro_setting'] == gyro_setting]
    df = df.sort_values(by=['filter_setting'])
    fig = px.histogram(df, x="gyr_X", y=df.index, color="filter_setting", marginal="rug", hover_data=df.columns)
    #update x axis title
    fig.update_xaxes(title_text='gyro x int 16')
    fig.update_yaxes(title_text='sum of samples')
    fig.update_layout(title_text='Histogram of gyro_X for gyro setting: ' + gyro_setting)
    fig.show()
    

In [ ]:
#plot the average of acc_X for each combination of acc setting and filter setting
df_all['acc_X'] = df_all['acc_X'].astype(float)
df_all['acc_Y'] = df_all['acc_Y'].astype(float)
df_all['acc_Z'] = df_all['acc_Z'].astype(float)
df_all['gyr_X'] = df_all['gyr_X'].astype(float)
df_all['gyr_Y'] = df_all['gyr_Y'].astype(float)
df_all['gyr_Z'] = df_all['gyr_Z'].astype(float)

df_mean = df_all.groupby(['acc_setting', 'filter_setting']).mean().reset_index()

#sort by acc_setting
df_mean['acc_setting'] = pd.Categorical(df_mean['acc_setting'], ['2G', '4G', '8G', '16G'])
df_mean = df_mean.sort_values('acc_setting')

# display(df_mean)

fig = px.line(df_mean, x='acc_setting', y='acc_X', color='filter_setting')
#set x axis title
fig.update_xaxes(title_text='Acceleration Setting')
fig.update_yaxes(title_text='Average Acceleration X INT16')
fig.show()

#add relative error to df_mean
# for 16 g setting 1g = 2048
# for 8 g setting 1g = 4096
# for 4 g setting 1g = 8192
# for 2 g setting 1g = 16384
# for each acc setting, calculate relative error
df_mean['relative_error'] = 0
for row in df_mean.itertuples():
    #get the value of 1g for this acc setting
    if row.acc_setting == '16G':
        one_g = 2048
    elif row.acc_setting == '8G':
        one_g = 4096
    elif row.acc_setting == '4G':
        one_g = 8192
    elif row.acc_setting == '2G':
        one_g = 16384
    else:
        print('error')
        break
    #calculate relative error
    relative_error = abs(row.acc_X - one_g) / one_g
    df_mean.at[row.Index, 'relative_error'] = relative_error

# for acc_setting in df_all['acc_setting'].unique():
#     for filter_setting in df_all['filter_setting'].unique():
#         display(acc_setting, filter_setting)
#         display(df_all[(df_all['acc_setting'] == acc_setting) & (df_all['filter_setting'] == filter_setting)])
#         mean_Acc_X = df_all[(df_all['acc_setting'] == acc_setting) & (df_all['filter_setting'] == filter_setting)]['acc_X'].mean()
#         print(mean_Acc_X)
#         break
#     break

In [ ]:
#plot the average of acc_X for each combination of acc setting and filter setting
df_all['acc_X'] = df_all['acc_X'].astype(float)
df_all['acc_Y'] = df_all['acc_Y'].astype(float)
df_all['acc_Z'] = df_all['acc_Z'].astype(float)
df_all['gyr_X'] = df_all['gyr_X'].astype(float)
df_all['gyr_Y'] = df_all['gyr_Y'].astype(float)
df_all['gyr_Z'] = df_all['gyr_Z'].astype(float)

df_mean = df_all.groupby(['acc_setting', 'filter_setting']).mean().reset_index()
#plot the relaive error for each combination of acc setting and filter setting
df_mean['acc_setting'] = pd.Categorical(df_mean['acc_setting'], ['2G', '4G', '8G', '16G'])
df_mean = df_mean.sort_values('acc_setting')

df_mean['relative_error'] = 0
for row in df_mean.itertuples():
    #get the value of 1g for this acc setting
    if row.acc_setting == '16G':
        one_g = 2048
    elif row.acc_setting == '8G':
        one_g = 4096
    elif row.acc_setting == '4G':
        one_g = 8192
    elif row.acc_setting == '2G':
        one_g = 16384
    else:
        print('error')
        break
    #calculate relative error
    relative_error = abs(row.acc_X - one_g) / one_g
    df_mean.at[row.Index, 'relative_error'] = relative_error

fig = px.line(df_mean, x='acc_setting', y='relative_error', color='filter_setting')
fig.update_xaxes(title_text='Acceleration Setting')
fig.update_yaxes(title_text='Relative Error')
fig.show()


In [ ]:
#show mean of acc_X for each acc setting and filter setting
# fig = px.bar(df_mean, x='acc_setting', y='acc_X', color='filter_setting')
# fig.update_xaxes(title_text='Acceleration Setting')
# fig.update_yaxes(title_text='Average Acceleration X INT16')
# fig.show()


df = df_all.copy()
df['acc_setting'] = pd.Categorical(df['acc_setting'], ['2G', '4G', '8G', '16G'])
df= df.sort_values('acc_setting')
df = df.sort_values(by=['acc_setting'])
fig = px.line(df, x="acc_setting", y="acc_X", color='filter_setting', hover_data=df.columns)
fig.update_xaxes(title_text='Acceleration Setting')
fig.update_yaxes(title_text='Acceleration X INT16')
fig.update_layout(title_text='Acceleration X with different filter setting MEAN int16')
fig.show()

# for filter_setting in df_all['filter_setting'].unique():
#     df = df_all[df_all['filter_setting'] == filter_setting]
#     df['acc_setting'] = pd.Categorical(df['acc_setting'], ['2G', '4G', '8G', '16G'])
#     df= df.sort_values('acc_setting')
#     df = df.sort_values(by=['acc_setting'])
#     fig = px.line(df, x="acc_setting", y="acc_X", hover_data=df.columns)
#     fig.update_xaxes(title_text='Acceleration Setting')
#     fig.update_yaxes(title_text='Acceleration X INT16')
#     fig.update_layout(title_text='Acceleration X for filter setting: ' + filter_setting)
#     fig.show()